In [1]:
import mne
import numpy as np
from npy_append_array import NpyAppendArray
import matplotlib.pyplot as plt
import pickle
import time
from sklearn.decomposition import PCA
import os

In [2]:
def pca(channel_data):
    number_of_data_points = len(channel_data)
    x = channel_data
    y = np.zeros(shape=(number_of_data_points - 1,))

    for i in range(1, len(x)):
        y[i-1] = x[i-1] - x[i]
    
    x = x[:-1]
    
    A = np.column_stack((x, y))
    pca = PCA(2)
    pca.fit(A)
    B = pca.transform(A)
    
    return B

In [3]:
class Seizure:
    def __init__(self):
        self.needed_epochs = []
        self.onset = 0
        
    def __str__(self):
        return f"needed_epochs: {self.needed_epochs}\nonset: {self.onset}\n"

# Classes Length : 30 min , Needed Interval : 300 min

In [2]:
channels = ["FP1-F7", "F7-T7", "T7-P7", "P7-O1", "FP1-F3", "F3-C3", 
            "C3-P3", "P3-O1", "FP2-F4", "F4-C4", "C4-P4", "P4-O2", 
            "FP2-F8", "F8-T8", "P8-O2", "FZ-CZ", "CZ-PZ"]

number_of_channels = len(channels)
interval = 300 * 60
classes_duration = 60 * 60
number_of_classes = interval // classes_duration
postictal_duration = 10 * 60
segment_size = 5

In [5]:
def get_datapoints(segment):
    
    data_points = np.zeros(shape=(len(channels), len(segment)))
    
    # Cleaning the raw data
    for i in range(len(channels)):
        data_points[i] = segment[i][0][0]
    
    return data_points

In [6]:
def define_classes_in_interval(raw_interval, pt_number, counter):
    start = 0
    end = segment_size
    init_counter = counter * (classes_duration//5)
    counter = init_counter
    _class = number_of_classes + 1
    
    while end < interval:
        current_segment_datapoints = get_datapoints(raw_interval.copy().crop(tmin=start, tmax=end, include_tmax=False))
        
        # Saving
        if start % classes_duration == 0:
            _class -= 1
            counter = init_counter
            
        np.save(f"Data/Interval-300-classes-length-{classes_duration//60}/data-points/class-{_class}/pt{pt_number[-2:]}-class{_class}-{counter}-{classes_duration//60}min.npy", current_segment_datapoints)
        
        start += segment_size
        end += segment_size
        counter += 1

In [7]:
def load_raw_eeg(epoch_names, onset):
    raw_data = []
    pt_number = epoch_names[0][:5]
    
    for epoch_name in epoch_names:
        raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
    
    try:
        all_raw_data = mne.concatenate_raws(raw_data,)
        
    except:
        all_raw_data = mne.concatenate_raws([raw.load_data(verbose=0).pick_channels(channels) for raw in raw_data])
    
    
    total_duration = len(all_raw_data)//256
    
    last_epoch_duration = len(raw_data[-1])//256
    
    interval_ending = (total_duration - last_epoch_duration) + onset
    interval_begining = interval_ending - interval
    
    raw_interval = all_raw_data.copy().crop(tmin=interval_begining, tmax=interval_ending, include_tmax=False)
    
    return raw_interval.load_data(verbose=0).pick_channels(channels), pt_number

## Loading Seizures Data

In [8]:
seizures = []

with open("Seizures Data.pkl", 'rb') as f:
    seizures = pickle.load(f) 

## Making Directories

In [6]:
dirs = ["data-points", "data-points-after-pca", "combined-channels-data", "cluster_centroids"]

In [7]:
# os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{dirs[0]}")
# os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{dirs[1]}")
os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{dirs[2]}")
os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{dirs[3]}")

In [8]:
for _dir in dirs[2:]:
    for _class in range(1, number_of_classes + 1):
        os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{_dir}/class-{_class}")

In [9]:
# for _class in range(1, number_of_classes + 1):    
#     os.makedirs(f"Data/Interval-300-classes-length-{classes_duration//60}/{dirs[2]}/class-{_class}")

## Converting Raw EEGs To Data Points

In [12]:
counter = 0

for seizure in seizures:
    t1 = time.time()
    raw_interval, pt_number = load_raw_eeg(seizure.needed_epochs[::-1], seizure.onset)
    define_classes_in_interval(raw_interval, pt_number, counter)
    
    print(seizure.needed_epochs[::-1])
    t2 = time.time()
    print(t2 - t1)
    counter += 1
    

<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-

['chb01_10.edf', 'chb01_11.edf', 'chb01_12.edf', 'chb01_13.edf', 'chb01_14.edf', 'chb01_15.edf']
830.3936710357666


<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-

['chb01_21.edf', 'chb01_22.edf', 'chb01_23.edf', 'chb01_24.edf', 'chb01_25.edf', 'chb01_26.edf']
795.2538945674896


<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))


...

['chb18_24.edf', 'chb18_25.edf', 'chb18_26.edf', 'chb18_27.edf', 'chb18_28.edf', 'chb18_29.edf']
768.8192284107208


<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Scaling factor is not defined in following channels:
--0, --1, --2, --3, --4
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_

['chb21_14.edf', 'chb21_15.edf', 'chb21_16.edf', 'chb21_17.edf', 'chb21_18.edf', 'chb21_19.edf']
775.9495768547058


<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.read_raw_edf(f"chb-mit-scalp-eeg-database-1.0.0/{pt_number}/{epoch_name}", verbose=0))
<ipython-input-7-482e06fa0986>:6: RuntimeWarning: Channel names are not unique, found duplicates for: {'-', 'T8-P8'}. Applying running numbers for duplicates.
  raw_data.append(mne.io.rea

['chb22_15.edf', 'chb22_16.edf', 'chb22_17.edf', 'chb22_18.edf', 'chb22_19.edf', 'chb22_20.edf']
822.5469214916229


In [3]:
datapoints_path = f"Data/Interval-300-classes-length-{classes_duration//60}/data-points"
datapoints_after_pca_path = f"Data/Interval-300-classes-length-{classes_duration//60}/data-points-after-pca"
combined_channels_data = f"Data/Interval-300-classes-length-{classes_duration//60}/combined-channels-data"

# Saving Data After Performing PCA

In [12]:
t1 = time.time()

for _class in range(1, number_of_classes + 1):
    current_class_path = f"{datapoints_path}/class-{_class}"
    current_class_list = os.listdir(current_class_path)
    
    for file in current_class_list:
        current_class_data = np.load(current_class_path + '/' + file)

        channels_data_after_pca = []
        for channel_data in current_class_data:
            pca_result = pca(channel_data)
            channels_data_after_pca.append(pca_result)

        np.save(f"{datapoints_after_pca_path}/class-{_class}/{file}", channels_data_after_pca)

t2 = time.time()
t2 - t1

5388.920633792877

# Combining All The Data For Each Channel

In [ ]:
for _class in range(1, number_of_classes + 1):   
    current_class_path = f"{datapoints_after_pca_path}/class-{_class}"
    current_class_list = os.listdir(current_class_path)

    for file in current_class_list:
        current_file = np.load(f"{current_class_path}/{file}")

        for i in range(number_of_channels):
            npa = NpyAppendArray(f"{combined_channels_data}/class-{_class}/channel_{i}.npy")
            npa.append(current_file[i])
            
    print('done')

done
done
done
done


In [13]:
for i in range(1):
    print(i)

0
